In [1]:
import pandas as pd
import datetime

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.stats.stattools import durbin_watson
# For error calculation
from sklearn.metrics import mean_squared_error, mean_absolute_error

import random
import numpy as np

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 500)

In [2]:
def filter_data(df_data, tickers, date, data_lookback = 60):
    # Function to calculate start date from a given date
    sdate = lambda x: (pd.to_datetime(x) - datetime.timedelta(days = 30 * data_lookback)).strftime('%Y%m%d')    
    #print(sdate(date))
    #print(datetime.timedelta(days = 30 * data_lookback))
    # Create subset for only the applicable tickers
    df_subset = df_data.loc[sdate(date):date][tickers]
    #print(df_subset)
    #Eliminate any tickers that have NaNs as some or all values
    for col in df_subset.columns:    
        # count NaN occurences     
        count_null = df_subset[col].isnull().sum()
        if count_null == 0:
            continue
        else:
            df_subset.drop(columns = col, inplace = True)
            #print('Dropped: ', col) 
            
    return df_subset

In [4]:
def train_model(X_train_transformed, X_test, n_obs = 5, verbose = False, maxlags=15, ic="aic", trend="ct"):
    
     # Train model
    model = VAR(X_train_transformed)
    res = model.fit(maxlags = maxlags, ic = ic, trend = trend)
    #res = model.fit(15)

    lag_order = res.k_ar

    if verbose == True:
        print("Lag samples to use for future predictions : ", lag_order)
        
    # Input data for forecasting
    input_data = X_train_transformed.values[-lag_order:]
    #print(input_data)
    # forecasting
    if lag_order > 0:
        pred = res.forecast(y=input_data, steps=n_obs)
        pred = (pd.DataFrame(pred, index=X_test.index, columns=X_test.columns + '_pred'))   

        # Save coefficients to their own df
        df_coef = pd.DataFrame(zip(X_train_transformed.columns, (res.coefs.mean(axis=2).flatten())), columns = ['ticker', 'coef']).sort_values('coef', ascending = False).reset_index(drop = True)

        return pred, df_coef, lag_order, res
    else:
        return None, None, None, None

In [5]:
def invert_transformation(X_train, pred):
    forecast = pred.copy()
    columns = X_train.columns
    for col in columns:
        forecast[str(col)+'_pred'] = X_train[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast

In [6]:
def make_predictions(X_train, pred, X_test):

    # Invert transform to get actual numerical predictions
    output = invert_transformation(X_train, pred)  

    #combining predicted and real data set
    combine_csi1_x = pd.concat([output['csi1_pred'], X_test['csi1']], axis=1)
    combine_csi1_x['accuracy'] = round(combine_csi1_x.apply(lambda row: row.csi1_pred /row.csi1 *100, axis = 1),2)
    combine_csi1_x['accuracy'] = pd.Series(["{0:.2f}%".format(val) for val in combine_csi1_x['accuracy']],index = combine_csi1_x.index)
    combine_csi1_x = combine_csi1_x.round(decimals=2).reset_index()
    combine_csi1_x = combine_csi1_x.sort_values(by='date', ascending=False)

    #Forecast bias
    forecast_errors = [combine_csi1_x['csi1'][i]- combine_csi1_x['csi1_pred'][i] for i in range(len(combine_csi1_x['csi1']))]
    bias = sum(forecast_errors) * 1.0/len(combine_csi1_x['csi1'])
    MSE = mean_squared_error(combine_csi1_x['csi1'].values, combine_csi1_x['csi1_pred'].values)
    #     print('Bias: %f' % bias)
    #     print('Mean absolute error:', mean_absolute_error(combine_csi1_x['csi1'].values, combine_csi1_x['csi1_pred'].values))
    #     print('Mean squared error:', MSE)
    #     print('Root mean squared error:', np.sqrt(mean_squared_error(combine_csi1_x['csi1'].values, combine_csi1_x['csi1_pred'].values)))

    return combine_csi1_x, MSE

In [7]:
# Setup one year of CSI prediction using all the possible methods 

# AIC, etc, etc, 
# Varying lag periods 

# Create ability to pull out Standard Errors if lag period chosen is greater than 1

csi_data = pd.read_csv("csi_data.csv").drop(columns=["csi2", "difference"])
tickers = pd.read_csv("SP500_tickers.csv")
stock_data = pd.read_csv("stock_data.csv")

tickers_final = [ticker for ticker in tickers.ticker if ticker in stock_data.columns]

print("List of Tickers in Stock Data : ", len(tickers_final))

stock_data =  stock_data[tickers_final]

df_data = pd.concat((csi_data, stock_data), axis=1)
print(df_data.shape)

# Bring in data
#df_data = pd.read_csv('_data/df_data_regular')
df_data['date'] = df_data.apply(lambda x: pd.to_datetime(x['date']), axis = 1)
df_data.set_index('date', inplace = True)

print(df_data.shape)

df_data.head()

List of Tickers in Stock Data :  471
(3523, 473)
(3523, 472)


,csi1,AAPL,MSFT,AMZN,FB,GOOGL,GOOG,JNJ,V,PG,JPM,UNH,HD,MA,NVDA,VZ,PYPL,PFE,NFLX,ADBE,T,DIS,INTC,MRK,CMCSA,CSCO,BAC,PEP,KO,WMT,ABT,XOM,CRM,ABBV,TMO,CVX,MCD,AMGN,COST,ACN,NEE,BMY,MDT,LLY,DHR,LIN,AVGO,PM,QCOM,NKE,TXN,UNP,AMT,ORCL,LOW,IBM,C,HON,UPS,LMT,WFC,GILD,AMD,FIS,SBUX,BLK,MMM,BA,CHTR,SPGI,CVS,NOW,BDX,ISRG,MDLZ,INTU,MO,PLD,CAT,VRTX,CCI,ZTS,ANTM,BKNG,EQIX,D,CI,CL,SYK,REGN,ATVI,APD,AXP,TGT,DUK,TJX,GS,TMUS,MMC,AMAT,MS,CB,SO,CME,ADP,FISV,MU,ILMN,DE,BSX,LRCX,GE,CSX,ITW,NEM,GPN,HUM,SHW,KMB,ICE,ADSK,PGR,NOC,USB,TFC,EW,DG,NSC,AON,ECL,MCO,PNC,BIIB,ROP,BAX,EL,ADI,DLR,WM,AEP,FDX,COP,EA,DXCM,DD,CNC,SCHW,GIS,ETN,CTSH,EMR,LHX,EXC,SRE,EBAY,XEL,ORLY,SBAC,GD,HCA,INFO,IDXX,MSCI,ROST,GM,KLAC,APH,DOW,TROW,ES,VRSK,STZ,SNPS,CDNS,WBA,WEC,TEL,A,COF,PSA,TRV,CMG,MET,MNST,RMD,PCAR,ALL,BK,CLX,TWTR,CMI,IQV,ZBH,AZO,JCI,KMI,YUM,PSX,AIG,KR,XLNX,SYY,WLTW,EOG,FAST,ANSS,AWK,F,PRU,ED,PPG,SLB,AFL,ROK,MCHP,CTAS,MCK,HPQ,MPC,ADM,BLL,GLW,SWK,MSI,SWKS,MKC,WMB,ALXN,MAR,PH,VLO,PAYX,WELL,BBY,STT,ARE,DLTR,DHI,CHD,DTE,FLT,MTD,CERN,VRSN,HSY,TDG,HLT,KHC,CTVA,FTV,ALGN,AME,AVB,WY,EIX,ETR,O,APTV,PPL,LEN,AJG,WST,EFX,SPG,MKTX,VFC,AEE,CPRT,FRC,LH,EQR,FCX,AMP,MXIM,KEYS,FTNT,RSG,TSN,CAG,HOLX,AKAM,TTWO,TFX,INCY,K,ODFL,TSCO,DGX,LYB,CDW,NTRS,CAH,LUV,CTXS,KMX,PXD,DFS,VMC,FE,LVS,ABC,BR,DOV,MAS,QRVO,KSU,NDAQ,ZBRA,GWW,DPZ,HIG,GRMN,ESS,DAL,VTR,HRL,EXPD,OXY,IP,PAYC,FITB,MTB,TYL,VIAC,COO,STE,FMC,JKHY,ANET,ABMD,TIF,XYL,MAA,IFF,WAT,EXR,GPC,PKI,MLM,NUE,HAL,BXP,CHRW,ATO,VAR,WDC,SYF,HPE,SJM,HES,LDOS,IEX,CINF,KEY,OKE,CE,EXPE,SIVB,OMC,ETFC,WAB,NLOK,BIO,ULTA,URI,TDY,PHM,IT,J,JBHT,IR,FBHS,AAP,CNP,RF,STX,EMN,DRI,WHR,AES,WU,CXO,HSIC,XRAY,BKR,NTAP,CBOE,HBAN,AVY,HAS,CTL,PNW,PKG,ALLE,NI,FOXA,WRB,LKQ,UHS,ALB,RE,LW,RCL,NRG,MGM,RJF,DISH,MYL,JNPR,CPB,DVA,TXT,IRM,FFIV,UAL,GL,SNA,HST,BWA,PRGO,COG,WYNN,LNC,WRK,HII,IPG,TAP,CCL,NWL,CF,LYV,IPGP,AIZ,FANG,APA,PWR,ZION,SEE,AAL,CMA,FLIR,LB,FRT,HBI,LEG,NLSN,PBCT,MHK,NBL,NWSA,MOS,DXC,MRO,NOV,ALK,DVN,HFC,SLG,IVZ,PVH,TPR,NCLH,RL,FTI,DISCA,KSS,XRX,GPS,HRB,UAA,UA,COTY
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-01-03,0.0,0.084282,0.211839,0.375533,NaN,1.719281,1.711147,0.486425,NaN,0.463931,0.317206,0.487214,0.325493,NaN,0.100552,0.215639,NaN,0.187687,0.029372,0.304025,0.195028,0.189981,0.201815,0.258485,0.069008,0.137727,0.371586,0.471665,0.161405,0.364878,0.149772,0.461484,0.067561,NaN,0.238990,0.466298,0.264562,0.634254,0.394002,0.231255,0.331965,0.183425,0.456590,0.452723,0.168624,0.424862,NaN,NaN,0.347277,0.084797,0.262352,0.158149,0.215470,0.099448,0.264680,0.647672,3.890292,0.281872,0.596290,0.501579,0.251776,0.107932,0.255722,0.323915,0.121823,0.860300,0.624388,0.555959,NaN,0.408998,0.207972,NaN,0.477111,0.304657,0.221389,0.213418,0.591792,0.392660,0.456196,0.220205,0.212313,NaN,0.626362,0.179006,0.326993,0.312391,0.295449,0.216693,0.352013,0.125493,0.055012,0.435228,0.414996,0.431886,0.383058,0.045679,1.017127,NaN,0.251618,0.144672,0.460221,0.430466,0.276796,0.573054,0.289997,0.086208,0.106630,0.058445,0.268350,0.193054,0.289582,0.268426,0.065706,0.353512,0.450987,0.187135,0.440253,0.356669,0.449647,0.059763,0.337411,0.227940,0.425223,0.238990,0.333938,0.027230,NaN,0.342936,0.294002,0.281768,0.496922,0.501657,0.368903,0.312155,0.164455,0.131571,0.289661,0.179558,0.242068,0.294870,0.815470,0.364071,0.420994,0.114444,0.493483,0.052190,0.118074,0.195659,0.265312,0.099980,0.296488,0.327213,0.431097,0.364167,0.147688,0.146567,0.252486,0.145383,0.453828,NaN,NaN,0.142798,NaN,0.058090,NaN,0.400079,0.087747,NaN,0.288043,0.156275,NaN,0.205130,0.161484,0.133781,0.358248,0.157459,NaN,0.189130,0.686504,0.552170,0.362983,NaN,0.355381,0.027269,0.15082

In [8]:
df_data.tail()

,csi1,AAPL,MSFT,AMZN,FB,GOOGL,GOOG,JNJ,V,PG,JPM,UNH,HD,MA,NVDA,VZ,PYPL,PFE,NFLX,ADBE,T,DIS,INTC,MRK,CMCSA,CSCO,BAC,PEP,KO,WMT,ABT,XOM,CRM,ABBV,TMO,CVX,MCD,AMGN,COST,ACN,NEE,BMY,MDT,LLY,DHR,LIN,AVGO,PM,QCOM,NKE,TXN,UNP,AMT,ORCL,LOW,IBM,C,HON,UPS,LMT,WFC,GILD,AMD,FIS,SBUX,BLK,MMM,BA,CHTR,SPGI,CVS,NOW,BDX,ISRG,MDLZ,INTU,MO,PLD,CAT,VRTX,CCI,ZTS,ANTM,BKNG,EQIX,D,CI,CL,SYK,REGN,ATVI,APD,AXP,TGT,DUK,TJX,GS,TMUS,MMC,AMAT,MS,CB,SO,CME,ADP,FISV,MU,ILMN,DE,BSX,LRCX,GE,CSX,ITW,NEM,GPN,HUM,SHW,KMB,ICE,ADSK,PGR,NOC,USB,TFC,EW,DG,NSC,AON,ECL,MCO,PNC,BIIB,ROP,BAX,EL,ADI,DLR,WM,AEP,FDX,COP,EA,DXCM,DD,CNC,SCHW,GIS,ETN,CTSH,EMR,LHX,EXC,SRE,EBAY,XEL,ORLY,SBAC,GD,HCA,INFO,IDXX,MSCI,ROST,GM,KLAC,APH,DOW,TROW,ES,VRSK,STZ,SNPS,CDNS,WBA,WEC,TEL,A,COF,PSA,TRV,CMG,MET,MNST,RMD,PCAR,ALL,BK,CLX,TWTR,CMI,IQV,ZBH,AZO,JCI,KMI,YUM,PSX,AIG,KR,XLNX,SYY,WLTW,EOG,FAST,ANSS,AWK,F,PRU,ED,PPG,SLB,AFL,ROK,MCHP,CTAS,MCK,HPQ,MPC,ADM,BLL,GLW,SWK,MSI,SWKS,MKC,WMB,ALXN,MAR,PH,VLO,PAYX,WELL,BBY,STT,ARE,DLTR,DHI,CHD,DTE,FLT,MTD,CERN,VRSN,HSY,TDG,HLT,KHC,CTVA,FTV,ALGN,AME,AVB,WY,EIX,ETR,O,APTV,PPL,LEN,AJG,WST,EFX,SPG,MKTX,VFC,AEE,CPRT,FRC,LH,EQR,FCX,AMP,MXIM,KEYS,FTNT,RSG,TSN,CAG,HOLX,AKAM,TTWO,TFX,INCY,K,ODFL,TSCO,DGX,LYB,CDW,NTRS,CAH,LUV,CTXS,KMX,PXD,DFS,VMC,FE,LVS,ABC,BR,DOV,MAS,QRVO,KSU,NDAQ,ZBRA,GWW,DPZ,HIG,GRMN,ESS,DAL,VTR,HRL,EXPD,OXY,IP,PAYC,FITB,MTB,TYL,VIAC,COO,STE,FMC,JKHY,ANET,ABMD,TIF,XYL,MAA,IFF,WAT,EXR,GPC,PKI,MLM,NUE,HAL,BXP,CHRW,ATO,VAR,WDC,SYF,HPE,SJM,HES,LDOS,IEX,CINF,KEY,OKE,CE,EXPE,SIVB,OMC,ETFC,WAB,NLOK,BIO,ULTA,URI,TDY,PHM,IT,J,JBHT,IR,FBHS,AAP,CNP,RF,STX,EMN,DRI,WHR,AES,WU,CXO,HSIC,XRAY,BKR,NTAP,CBOE,HBAN,AVY,HAS,CTL,PNW,PKG,ALLE,NI,FOXA,WRB,LKQ,UHS,ALB,RE,LW,RCL,NRG,MGM,RJF,DISH,MYL,JNPR,CPB,DVA,TXT,IRM,FFIV,UAL,GL,SNA,HST,BWA,PRGO,COG,WYNN,LNC,WRK,HII,IPG,TAP,CCL,NWL,CF,LYV,IPGP,AIZ,FANG,APA,PWR,ZION,SEE,AAL,CMA,FLIR,LB,FRT,HBI,LEG,NLSN,PBCT,MHK,NBL,NWSA,MOS,DXC,MRO,NOV,ALK,DVN,HFC,SLG,IVZ,PVH,TPR,NCLH,RL,FTI,DISCA,KSS,XRX,GPS,HRB,UAA,UA,COTY
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-24,1.437937,0.884942,0.489929,5.569872,0.638546,4.185257,4.182548,0.454285,0.583912,0.389814,0.428291,0.916913,0.686953,0.926065,0.742832,0.190767,0.338356,0.122311,1.037263,1.026181,0.121284,0.452293,0.184945,0.284656,0.138623,0.148741,0.109641,0.426143,0.170314,0.372039,0.271706,0.217975,0.508203,0.279706,1.016437,0.374623,0.612240,0.754382,0.915948,0.658749,0.748716,0.196619,0.354170,0.408150,0.477353,0.658780,0.997727,0.268063,0.276344,0.311739,0.400118,0.560004,0.706908,0.166361,0.372848,0.420197,0.244653,0.548859,0.368365,1.214613,0.167543,0.207702,0.144881,0.429225,0.275566,1.555396,0.550011,1.036640,1.496934,0.848302,0.231952,0.884724,0.849889,1.864116,0.171186,0.822868,0.156523,0.273511,0.459110,0.686611,0.435949,0.413784,0.947919,6.363042,1.784516,0.253276,0.635090,0.213399,0.658656,1.172089,0.183078,0.731719,0.388320,0.400181,0.280889,0.187965,0.715718,0.240700,0.347321,0.191452,0.157800,0.479220,0.196526,0.633067,0.528936,0.363073,0.172524,1.044392,0.543224,0.141052,0.916726,0.034835,0.227189,0.560906,0.131308,0.569187,1.146655,1.809109,0.427202,0.286991,0.572518,0.223734,1.080472,0.185755,0.174828,0.244674,0.486412,0.606762,0.646577,0.595990,0.741462,0.499393,0.946580,1.099648,0.263643,0.639573,0.372692,0.363416,0.350528,0.292127,0.469383,0.201320,0.337920,0.673661,0.198549,0.196837,0.151636,0.164929,0.294119,0.194253,0.236902,0.626405,0.141083,0.468979,0.112723,0.195685,1.369206,0.747595,0.556673,0.460667,0.233882,0.819537,0.810510,0.358373,0.113875,0.554463,0.333811,0.170906,0.380413,0.261370,0.464371,0.581608,0.430252,0

In [15]:
total_snp_tickers = df_data.columns[1:].tolist()

print(total_snp_tickers)

['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JNJ', 'V', 'PG', 'JPM', 'UNH', 'HD', 'MA', 'NVDA', 'VZ', 'PYPL', 'PFE', 'NFLX', 'ADBE', 'T', 'DIS', 'INTC', 'MRK', 'CMCSA', 'CSCO', 'BAC', 'PEP', 'KO', 'WMT', 'ABT', 'XOM', 'CRM', 'ABBV', 'TMO', 'CVX', 'MCD', 'AMGN', 'COST', 'ACN', 'NEE', 'BMY', 'MDT', 'LLY', 'DHR', 'LIN', 'AVGO', 'PM', 'QCOM', 'NKE', 'TXN', 'UNP', 'AMT', 'ORCL', 'LOW', 'IBM', 'C', 'HON', 'UPS', 'LMT', 'WFC', 'GILD', 'AMD', 'FIS', 'SBUX', 'BLK', 'MMM', 'BA', 'CHTR', 'SPGI', 'CVS', 'NOW', 'BDX', 'ISRG', 'MDLZ', 'INTU', 'MO', 'PLD', 'CAT', 'VRTX', 'CCI', 'ZTS', 'ANTM', 'BKNG', 'EQIX', 'D', 'CI', 'CL', 'SYK', 'REGN', 'ATVI', 'APD', 'AXP', 'TGT', 'DUK', 'TJX', 'GS', 'TMUS', 'MMC', 'AMAT', 'MS', 'CB', 'SO', 'CME', 'ADP', 'FISV', 'MU', 'ILMN', 'DE', 'BSX', 'LRCX', 'GE', 'CSX', 'ITW', 'NEM', 'GPN', 'HUM', 'SHW', 'KMB', 'ICE', 'ADSK', 'PGR', 'NOC', 'USB', 'TFC', 'EW', 'DG', 'NSC', 'AON', 'ECL', 'MCO', 'PNC', 'BIIB', 'ROP', 'BAX', 'EL', 'ADI', 'DLR', 'WM', 'AEP', 'FDX', 'COP', 'E

In [16]:
idxs = list(range(0, len(total_snp_tickers), 45)) + [len(total_snp_tickers)]

final_combinations = list(zip(idxs[:-1], idxs[1:]))

final_combinations

[(0, 45),
 (45, 90),
 (90, 135),
 (135, 180),
 (180, 225),
 (225, 270),
 (270, 315),
 (315, 360),
 (360, 405),
 (405, 450),
 (450, 471)]

In [10]:
# Below is a method for pulling out std errors from the model results

#variable_of_interest = 'csi1'
#qty_tickers = 40

# Save all 2012 dates to a list to refer to later 
#dates = [x.strftime('%Y%m%d') for x in list(df_data.loc[pd.to_datetime('20120101'):pd.to_datetime('20121231')].index)]
#print(len(dates))
# Define tickers to use
#tickers = cols_x[:qty_tickers]

#print(tickers)
#for i in range(0, qty_tickers, 1):
#    tickers.append(random.choice(df_data.columns))
# Ensure the variable_of_interest is in the ticker list:
#tickers.append(variable_of_interest)
# Delete any duplicates
#tickers = list(set( tickers ))

def try_param_settings(df_data, tickers, date):
    ics = ['aic', 'fpe', 'hqic', 'bic']
    trends = ["c", "ct", "ctt", "nc", "n"]
    mls = range(5,20, 5)

    outputs, mses, stderrs, params = [], [], [], []
    
    ## Looping through various param settings
    for trend in trends:
        for ic in ics:
            for maxlags in mls:
                print("Trend : %s, IC : %s, Maxlag : %d"%(trend, ic, maxlags))
                # Create data subset 
                df_subset = filter_data(df_data, tickers, date)

                # "Difference" the data (statistics term)
                n_obs = 5 ## Beyond 5 results detriorate for majority of predictions

                # Isolate usable columns and ensure csi1 and csi_difference are in the final list
                final_cols = df_subset.columns 
                # Create training and test data
                X_train, X_test = df_subset[final_cols][0:-n_obs], df_subset[final_cols][-n_obs:]
                X_train_transformed = X_train.diff().dropna()
                
                # Train model and return predictions and coefficients
                pred, df_coef, lag_order, res = train_model(X_train_transformed, 
                                                            X_test,
                                                            n_obs, 
                                                            verbose = True,
                                                            maxlags=maxlags,
                                                            trend=trend,
                                                            ic=ic)

                # Output forward-predictions and MSE
                if isinstance(pred, pd.DataFrame):
                    output, MSE = make_predictions(X_train, pred, X_test)   
                else:
                    output, MSE = None, None


                params.append((trend, ic, maxlags))
                outputs.append(output)
                mses.append(MSE)

                if isinstance(pred, pd.DataFrame):
                    stderrs.append(res.stderr[["csi1"]].T)
                else:
                    stderrs.append(None)

                    
    return outputs, mses, stderrs, params
                #print(pred)
                #print(output)
                #print(res.stderr[["csi1"]].T)
                    # Isolate coefficients in relation to the variable of interest
                    #c = (pd.DataFrame(res.coefs[0]).loc[0]).values # Not 100% sure this is accurate

                    # Isolate std_err in relation to the variable of interest
                    #se = (pd.DataFrame(res.stderr)[variable_of_interest].iloc[2:]).values # Sometimes this needs to be .iloc[1:] and other times it needs to be .iloc[2:] - depends on whether AIC or HQIC is used, etc

                    # Create an index based on each ticker name
                    #index = [x[3:] for x in pd.DataFrame(res.stderr)[variable_of_interest].index[2:]]

                    # Create a dataframe using the above information 
                    #df_se = pd.DataFrame(se, index = index, columns = ['std_err'])

                    # Sort the dataframe by lowest standard error and drop the variable of interest
                    #df_se = df_se.sort_values('std_err')\
                    #    .drop(variable_of_interest).reset_index()\
                    #    .rename(columns = {'index' : 'ticker'})

                    #df_se
date = '20191231'

final_outputs, final_mses, final_stderrs, final_params = [], [], [], []

## Looping through 46 ticker combinations
for idxs in final_combinations:
    tickers = ["csi1"] + total_snp_tickers[idxs[0]:idxs[1]]
    print("\nTrying Tickers : ", tickers, "\n")
    
    outputs, mses, stderrs, params = try_param_settings(df_data, tickers, date)
    
    final_outputs.append(outputs)
    final_mses.append(mses)
    final_stderrs.append(stderrs)
    final_params.append(params)


Trying Tickers :  ['csi1', 'AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JNJ', 'V', 'PG', 'JPM', 'UNH', 'HD', 'MA', 'NVDA', 'VZ', 'PYPL', 'PFE', 'NFLX', 'ADBE', 'T', 'DIS', 'INTC', 'MRK', 'CMCSA', 'CSCO', 'BAC', 'PEP', 'KO', 'WMT', 'ABT', 'XOM', 'CRM', 'ABBV', 'TMO', 'CVX', 'MCD', 'AMGN', 'COST', 'ACN', 'NEE', 'BMY', 'MDT', 'LLY', 'DHR', 'LIN'] 

Trend : c, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : c, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : c, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : c, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : c, IC : fpe, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : c, IC : fpe, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : c, IC : hqic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : c, IC : hqic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : c

Lag samples to use for future predictions :  0
Trend : ctt, IC : bic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : nc, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : nc, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : nc, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : nc, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : nc, IC : fpe, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : nc, IC : fpe, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : nc, IC : hqic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : nc, IC : hqic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : nc, IC : hqic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : nc, IC : bic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : nc, IC : bic, Maxlag : 10
Lag samples to use for futur

Lag samples to use for future predictions :  0
Trend : c, IC : bic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : c, IC : bic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : c, IC : bic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : ct, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : ct, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : ct, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : ct, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : ct, IC : fpe, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : ct, IC : fpe, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : ct, IC : hqic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : ct, IC : hqic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : ct, IC : hqic, Maxlag : 15
Lag samples to use for future pr

Lag samples to use for future predictions :  0
Trend : nc, IC : bic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : n, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : n, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : n, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : n, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : n, IC : fpe, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : n, IC : fpe, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : n, IC : hqic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : n, IC : hqic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : n, IC : hqic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : n, IC : bic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : n, IC : bic, Maxlag : 10
Lag samples to use for future prediction

Lag samples to use for future predictions :  0
Trend : ct, IC : hqic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : ct, IC : bic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : ct, IC : bic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : ct, IC : bic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : ctt, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : ctt, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : ctt, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : ctt, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : ctt, IC : fpe, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : ctt, IC : fpe, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : ctt, IC : hqic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : ctt, IC : hqic, Maxlag : 10
Lag samples to use fo

Lag samples to use for future predictions :  0
Trend : n, IC : bic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : n, IC : bic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : n, IC : bic, Maxlag : 15
Lag samples to use for future predictions :  0

Trying Tickers :  ['csi1', 'NLOK', 'BIO', 'ULTA', 'URI', 'TDY', 'PHM', 'IT', 'J', 'JBHT', 'IR', 'FBHS', 'AAP', 'CNP', 'RF', 'STX', 'EMN', 'DRI', 'WHR', 'AES', 'WU', 'CXO', 'HSIC', 'XRAY', 'BKR', 'NTAP', 'CBOE', 'HBAN', 'AVY', 'HAS', 'CTL', 'PNW', 'PKG', 'ALLE', 'NI', 'FOXA', 'WRB', 'LKQ', 'UHS', 'ALB', 'RE', 'LW', 'RCL', 'NRG', 'MGM', 'RJF'] 

Trend : c, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : c, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : c, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : c, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : c, IC : fpe, Maxlag : 10
Lag samples to u

Lag samples to use for future predictions :  1
Trend : ctt, IC : hqic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : ctt, IC : hqic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : ctt, IC : hqic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : ctt, IC : bic, Maxlag : 5
Lag samples to use for future predictions :  0
Trend : ctt, IC : bic, Maxlag : 10
Lag samples to use for future predictions :  0
Trend : ctt, IC : bic, Maxlag : 15
Lag samples to use for future predictions :  0
Trend : nc, IC : aic, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : nc, IC : aic, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : nc, IC : aic, Maxlag : 15
Lag samples to use for future predictions :  1
Trend : nc, IC : fpe, Maxlag : 5
Lag samples to use for future predictions :  1
Trend : nc, IC : fpe, Maxlag : 10
Lag samples to use for future predictions :  1
Trend : nc, IC : fpe, Maxlag : 15
Lag samples to use for 

In [19]:
err_dfs = []
for stderrs in final_stderrs:
    first = stderrs[0]
    if "trend" in first.columns:
        first = first.drop(columns=["trend"])
    if "const" in first.columns:
        first = first.drop(columns=["const"])
    first = first.drop(columns=["L1.csi1"])
    #print(first.columns)
    
    err_data = []
    for stderr in stderrs:
        if isinstance(stderr, pd.DataFrame):
            err_data.append(stderr[first.columns].values.flatten().tolist())
        else:
            err_data.append([None]*first.shape[1])
    
    df  = pd.DataFrame(err_data, columns=first.columns)
    err_dfs.append(df)    
    
all_ticker_errors_df = pd.concat(err_dfs, axis=1)
all_ticker_errors_df.head()

,L1.AAPL,L1.MSFT,L1.AMZN,L1.FB,L1.GOOGL,L1.GOOG,L1.JNJ,L1.V,L1.PG,L1.JPM,L1.UNH,L1.HD,L1.MA,L1.NVDA,L1.VZ,L1.PFE,L1.NFLX,L1.ADBE,L1.T,L1.DIS,L1.INTC,L1.MRK,L1.CMCSA,L1.CSCO,L1.BAC,L1.PEP,L1.KO,L1.WMT,L1.ABT,L1.XOM,L1.CRM,L1.ABBV,L1.TMO,L1.CVX,L1.MCD,L1.AMGN,L1.COST,L1.ACN,L1.NEE,L1.BMY,L1.MDT,L1.LLY,L1.DHR,L1.LIN,L1.AVGO,L1.PM,L1.QCOM,L1.NKE,L1.TXN,L1.UNP,L1.AMT,L1.ORCL,L1.LOW,L1.IBM,L1.C,L1.HON,L1.UPS,L1.LMT,L1.WFC,L1.GILD,L1.AMD,L1.FIS,L1.SBUX,L1.BLK,L1.MMM,L1.BA,L1.CHTR,L1.SPGI,L1.CVS,L1.NOW,L1.BDX,L1.ISRG,L1.MDLZ,L1.INTU,L1.MO,L1.PLD,L1.CAT,L1.VRTX,L1.CCI,L1.ZTS,L1.ANTM,L1.BKNG,L1.EQIX,L1.D,L1.CI,L1.CL,L1.SYK,L1.REGN,L1.ATVI,L1.APD,L1.AXP,L1.TGT,L1.DUK,L1.TJX,L1.GS,L1.TMUS,L1.MMC,L1.AMAT,L1.MS,L1.CB,L1.SO,L1.CME,L1.ADP,L1.FISV,L1.MU,L1.ILMN,L1.DE,L1.BSX,L1.LRCX,L1.GE,L1.CSX,L1.ITW,L1.NEM,L1.GPN,L1.HUM,L1.SHW,L1.KMB,L1.ICE,L1.ADSK,L1.PGR,L1.NOC,L1.USB,L1.TFC,L1.EW,L1.DG,L1.NSC,L1.AON,L1.ECL,L1.MCO,L1.PNC,L1.BIIB,L1.ROP,L1.BAX,L1.EL,L1.ADI,L1.DLR,L1.WM,L1.AEP,L1.FDX,L1.COP,L1.EA,L1.DXCM,L1.DD,L1.CNC,L1.SCHW,L1.GIS,L1.ETN,L1.CTSH,L1.EMR,L1.LHX,L1.EXC,L1.SRE,L1.EBAY,L1.XEL,L1.ORLY,L1.SBAC,L1.GD,L1.HCA,L1.INFO,L1.IDXX,L1.MSCI,L1.ROST,L1.GM,L1.KLAC,L1.APH,L1.TROW,L1.ES,L1.VRSK,L1.STZ,L1.SNPS,L1.CDNS,L1.WBA,L1.WEC,L1.TEL,L1.A,L1.COF,L1.PSA,L1.TRV,L1.CMG,L1.MET,L1.MNST,L1.RMD,L1.PCAR,L1.ALL,L1.BK,L1.CLX,L1.TWTR,L1.CMI,L1.IQV,L1.ZBH,L1.AZO,L1.JCI,L1.KMI,L1.YUM,L1.PSX,L1.AIG,L1.KR,L1.XLNX,L1.SYY,L1.WLTW,L1.EOG,L1.FAST,L1.ANSS,L1.AWK,L1.F,L1.PRU,L1.ED,L1.PPG,L1.SLB,L1.AFL,L1.ROK,L1.MCHP,L1.CTAS,L1.MCK,L1.HPQ,L1.MPC,L1.ADM,L1.BLL,L1.GLW,L1.SWK,L1.MSI,L1.SWKS,L1.MKC,L1.WMB,L1.ALXN,L1.MAR,L1.PH,L1.VLO,L1.PAYX,L1.WELL,L1.BBY,L1.STT,L1.ARE,L1.DLTR,L1.DHI,L1.CHD,L1.DTE,L1.FLT,L1.MTD,L1.CERN,L1.VRSN,L1.HSY,L1.TDG,L1.HLT,L1.ALGN,L1.AME,L1.AVB,L1.WY,L1.EIX,L1.ETR,L1.O,L1.APTV,L1.PPL,L1.LEN,L1.AJG,L1.WST,L1.EFX,L1.SPG,L1.MKTX,L1.VFC,L1.AEE,L1.CPRT,L1.FRC,L1.LH,L1.EQR,L1.FCX,L1.AMP,L1.MXIM,L1.KEYS,L1.FTNT,L1.RSG,L1.TSN,L1.CAG,L1.HOLX,L1.AKAM,L1.TTWO,L1.TFX,L1.INCY,L1.K,L1.ODFL,L1.TSCO,L1.DGX,L1.LYB,L1.CDW,L1.NTRS,L1.CAH,L1.LUV,L1.CTXS,L1.KMX,L1.PXD,L1.DFS,L1.VMC,L1.FE,L1.LVS,L1.ABC,L1.BR,L1.DOV,L1.MAS,L1.QRVO,L1.KSU,L1.NDAQ,L1.ZBRA,L1.GWW,L1.DPZ,L1.HIG,L1.GRMN,L1.ESS,L1.DAL,L1.VTR,L1.HRL,L1.EXPD,L1.OXY,L1.IP,L1.PAYC,L1.FITB,L1.MTB,L1.TYL,L1.COO,L1.STE,L1.FMC,L1.JKHY,L1.ANET,L1.ABMD,L1.TIF,L1.XYL,L1.MAA,L1.IFF,L1.WAT,L1.EXR,L1.GPC,L1.PKI,L1.MLM,L1.NUE,L1.HAL,L1.BXP,L1.CHRW,L1.ATO,L1.VAR,L1.WDC,L1.SYF,L1.SJM,L1.HES,L1.LDOS,L1.IEX,L1.CINF,L1.KEY,L1.OKE,L1.CE,L1.EXPE,L1.SIVB,L1.OMC,L1.ETFC,L1.WAB,L1.BIO,L1.ULTA,L1.URI,L1.TDY,L1.PHM,L1.IT,L1.J,L1.JBHT,L1.FBHS,L1.AAP,L1.CNP,L1.RF,L1.STX,L1.EMN,L1.DRI,L1.WHR,L1.AES,L1.WU,L1.CXO,L1.HSIC,L1.XRAY,L1.BKR,L1.NTAP,L1.CBOE,L1.HBAN,L1.AVY,L1.HAS,L1.CTL,L1.PNW,L1.PKG,L1.ALLE,L1.NI,L1.WRB,L1.LKQ,L1.UHS,L1.ALB,L1.RE,L1.RCL,L1.NRG,L1.MGM,L1.RJF,L1.DISH,L1.MYL,L1.JNPR,L1.CPB,L1.DVA,L1.TXT,L1.IRM,L1.FFIV,L1.UAL,L1.GL,L1.SNA,L1.HST,L1.BWA,L1.PRGO,L1.COG,L1.WYNN,L1.LNC,L1.HII,L1.IPG,L1.TAP,L1.CCL,L1.NWL,L1.CF,L1.LYV,L1.IPGP,L1.AIZ,L1.FANG,L1.APA,L1.PWR,L1.ZION,L1.SEE,L1.AAL,L1.CMA,L1.FLIR,L1.LB,L1.FRT,L1.HBI,L1.LEG,L1.NLSN,L1.PBCT,L1.MHK,L1.NBL,L1.NWSA,L1.MOS,L1.DXC,L1.MRO,L1.NOV,L1.ALK,L1.DVN,L1.HFC,L1.SLG,L1.IVZ,L1.PVH,L1.TPR,L1.NCLH,L1.RL,L1.FTI,L1.DISCA,L1.KSS,L1.XRX,L1.GPS,L1.HRB,L1.UAA,L1.COTY
0,0.425819,1.166547,0.05781,0.366859,0.4133,0.415332,0.737451,1.10798,1.07054,1.489626,0.340135,0.519799,0.729145,0.24969,1.757723,2.384511,0.17346,0.418277,2.349222,0.629841,1.406323,1.256391,1.936607,1.909782,4.600287,1.136136,2.497979,0.797371,1.53489,1.12883,0.565797,0.615283,0.44851,0.745636,0.517061,0.399052,0.394416,0.637396,0.569783,0.81108,0.963316,0.689431,1.000856,0.532943,0.220264,0.784741,0.654079,0.867787,0.759656,0.534515,0.663593,1.531352,0.672226,0.466154,1.486465,0.833829,0.728019,0.27825,1.622394,0.615701,1.516479,0.820995,1.100814,0.213953,0.452035,0.216219,0.161522,0.511854,0.68759,0.313751,0.418586,0.166358,1.549911,0.428062,1.244998,1.329668,0.568551,0.294499,0.900643,0.926982,0.308598,0.032243,0.16

In [20]:
### Here MSEs_1 represents first 46 ticker MSE for that param combinations, MSEs_2 represents next 46 tickers.
## Actual tickers tried are 45 + 1 csi1. It goes till 11 because df_data has 471 tickers (45* 10 + 21)

MSEs = pd.DataFrame(data=final_mses).T
MSEs.columns = ["MSEs_%d"%(col) for col in MSEs.columns]
params_df = pd.DataFrame(data=final_params[0], columns=["Trend", "IC", "Maxlags"])

final_df = pd.concat((params_df, MSEs), axis=1)
final_df.head()

,Trend,IC,Maxlags,MSEs_0,MSEs_1,MSEs_2,MSEs_3,MSEs_4,MSEs_5,MSEs_6,MSEs_7,MSEs_8,MSEs_9,MSEs_10
0,c,aic,5,0.00492,0.01342,0.00498,0.00238,0.00256,0.00102,0.00636,0.0006,0.00238,0.00176,0.0033
1,c,aic,10,0.00492,0.01342,0.00498,0.00238,0.00256,0.00102,0.00636,0.0006,0.00238,0.00176,0.0033
2,c,aic,15,0.00492,0.01342,0.00498,0.00238,0.00256,0.00102,0.00636,0.0006,0.00238,0.00176,0.0033
3,c,fpe,5,0.00492,0.01342,0.00498,0.00238,0.00256,0.00102,0.00636,0.0006,0.00238,0.00176,0.0033
4,c,fpe,10,0.00492,0.01342,0.00498,0.00238,0.00256,0.00102,0.00636,0.0006,0.00238,0.00176,0.0033


In [21]:
# res.df_model indicates how much depth the VAR algo utilized... derived from maxlags = 15
# 525 / 35 = 15

final_df = pd.concat((final_df, all_ticker_errors_df), axis=1)
final_df

,Trend,IC,Maxlags,MSEs_0,MSEs_1,MSEs_2,MSEs_3,MSEs_4,MSEs_5,MSEs_6,MSEs_7,MSEs_8,MSEs_9,MSEs_10,L1.AAPL,L1.MSFT,L1.AMZN,L1.FB,L1.GOOGL,L1.GOOG,L1.JNJ,L1.V,L1.PG,L1.JPM,L1.UNH,L1.HD,L1.MA,L1.NVDA,L1.VZ,L1.PFE,L1.NFLX,L1.ADBE,L1.T,L1.DIS,L1.INTC,L1.MRK,L1.CMCSA,L1.CSCO,L1.BAC,L1.PEP,L1.KO,L1.WMT,L1.ABT,L1.XOM,L1.CRM,L1.ABBV,L1.TMO,L1.CVX,L1.MCD,L1.AMGN,L1.COST,L1.ACN,L1.NEE,L1.BMY,L1.MDT,L1.LLY,L1.DHR,L1.LIN,L1.AVGO,L1.PM,L1.QCOM,L1.NKE,L1.TXN,L1.UNP,L1.AMT,L1.ORCL,L1.LOW,L1.IBM,L1.C,L1.HON,L1.UPS,L1.LMT,L1.WFC,L1.GILD,L1.AMD,L1.FIS,L1.SBUX,L1.BLK,L1.MMM,L1.BA,L1.CHTR,L1.SPGI,L1.CVS,L1.NOW,L1.BDX,L1.ISRG,L1.MDLZ,L1.INTU,L1.MO,L1.PLD,L1.CAT,L1.VRTX,L1.CCI,L1.ZTS,L1.ANTM,L1.BKNG,L1.EQIX,L1.D,L1.CI,L1.CL,L1.SYK,L1.REGN,L1.ATVI,L1.APD,L1.AXP,L1.TGT,L1.DUK,L1.TJX,L1.GS,L1.TMUS,L1.MMC,L1.AMAT,L1.MS,L1.CB,L1.SO,L1.CME,L1.ADP,L1.FISV,L1.MU,L1.ILMN,L1.DE,L1.BSX,L1.LRCX,L1.GE,L1.CSX,L1.ITW,L1.NEM,L1.GPN,L1.HUM,L1.SHW,L1.KMB,L1.ICE,L1.ADSK,L1.PGR,L1.NOC,L1.USB,L1.TFC,L1.EW,L1.DG,L1.NSC,L1.AON,L1.ECL,L1.MCO,L1.PNC,L1.BIIB,L1.ROP,L1.BAX,L1.EL,L1.ADI,L1.DLR,L1.WM,L1.AEP,L1.FDX,L1.COP,L1.EA,L1.DXCM,L1.DD,L1.CNC,L1.SCHW,L1.GIS,L1.ETN,L1.CTSH,L1.EMR,L1.LHX,L1.EXC,L1.SRE,L1.EBAY,L1.XEL,L1.ORLY,L1.SBAC,L1.GD,L1.HCA,L1.INFO,L1.IDXX,L1.MSCI,L1.ROST,L1.GM,L1.KLAC,L1.APH,L1.TROW,L1.ES,L1.VRSK,L1.STZ,L1.SNPS,L1.CDNS,L1.WBA,L1.WEC,L1.TEL,L1.A,L1.COF,L1.PSA,L1.TRV,L1.CMG,L1.MET,L1.MNST,L1.RMD,L1.PCAR,L1.ALL,L1.BK,L1.CLX,L1.TWTR,L1.CMI,L1.IQV,L1.ZBH,L1.AZO,L1.JCI,L1.KMI,L1.YUM,L1.PSX,L1.AIG,L1.KR,L1.XLNX,L1.SYY,L1.WLTW,L1.EOG,L1.FAST,L1.ANSS,L1.AWK,L1.F,L1.PRU,L1.ED,L1.PPG,L1.SLB,L1.AFL,L1.ROK,L1.MCHP,L1.CTAS,L1.MCK,L1.HPQ,L1.MPC,L1.ADM,L1.BLL,L1.GLW,L1.SWK,L1.MSI,L1.SWKS,L1.MKC,L1.WMB,L1.ALXN,L1.MAR,L1.PH,L1.VLO,L1.PAYX,L1.WELL,L1.BBY,L1.STT,L1.ARE,L1.DLTR,L1.DHI,L1.CHD,L1.DTE,L1.FLT,L1.MTD,L1.CERN,L1.VRSN,L1.HSY,L1.TDG,L1.HLT,L1.ALGN,L1.AME,L1.AVB,L1.WY,L1.EIX,L1.ETR,L1.O,L1.APTV,L1.PPL,L1.LEN,L1.AJG,L1.WST,L1.EFX,L1.SPG,L1.MKTX,L1.VFC,L1.AEE,L1.CPRT,L1.FRC,L1.LH,L1.EQR,L1.FCX,L1.AMP,L1.MXIM,L1.KEYS,L1.FTNT,L1.RSG,L1.TSN,L1.CAG,L1.HOLX,L1.AKAM,L1.TTWO,L1.TFX,L1.INCY,L1.K,L1.ODFL,L1.TSCO,L1.DGX,L1.LYB,L1.CDW,L1.NTRS,L1.CAH,L1.LUV,L1.CTXS,L1.KMX,L1.PXD,L1.DFS,L1.VMC,L1.FE,L1.LVS,L1.ABC,L1.BR,L1.DOV,L1.MAS,L1.QRVO,L1.KSU,L1.NDAQ,L1.ZBRA,L1.GWW,L1.DPZ,L1.HIG,L1.GRMN,L1.ESS,L1.DAL,L1.VTR,L1.HRL,L1.EXPD,L1.OXY,L1.IP,L1.PAYC,L1.FITB,L1.MTB,L1.TYL,L1.COO,L1.STE,L1.FMC,L1.JKHY,L1.ANET,L1.ABMD,L1.TIF,L1.XYL,L1.MAA,L1.IFF,L1.WAT,L1.EXR,L1.GPC,L1.PKI,L1.MLM,L1.NUE,L1.HAL,L1.BXP,L1.CHRW,L1.ATO,L1.VAR,L1.WDC,L1.SYF,L1.SJM,L1.HES,L1.LDOS,L1.IEX,L1.CINF,L1.KEY,L1.OKE,L1.CE,L1.EXPE,L1.SIVB,L1.OMC,L1.ETFC,L1.WAB,L1.BIO,L1.ULTA,L1.URI,L1.TDY,L1.PHM,L1.IT,L1.J,L1.JBHT,L1.FBHS,L1.AAP,L1.CNP,L1.RF,L1.STX,L1.EMN,L1.DRI,L1.WHR,L1.AES,L1.WU,L1.CXO,L1.HSIC,L1.XRAY,L1.BKR,L1.NTAP,L1.CBOE,L1.HBAN,L1.AVY,L1.HAS,L1.CTL,L1.PNW,L1.PKG,L1.ALLE,L1.NI,L1.WRB,L1.LKQ,L1.UHS,L1.ALB,L1.RE,L1.RCL,L1.NRG,L1.MGM,L1.RJF,L1.DISH,L1.MYL,L1.JNPR,L1.CPB,L1.DVA,L1.TXT,L1.IRM,L1.FFIV,L1.UAL,L1.GL,L1.SNA,L1.HST,L1.BWA,L1.PRGO,L1.COG,L1.WYNN,L1.LNC,L1.HII,L1.IPG,L1.TAP,L1.CCL,L1.NWL,L1.CF,L1.LYV,L1.IPGP,L1.AIZ,L1.FANG,L1.APA,L1.PWR,L1.ZION,L1.SEE,L1.AAL,L1.CMA,L1.FLIR,L1.LB,L1.FRT,L1.HBI,L1.LEG,L1.NLSN,L1.PBCT,L1.MHK,L1.NBL,L1.NWSA,L1.MOS,L1.DXC,L1.MRO,L1.NOV,L1.ALK,L1.DVN,L1.HFC,L1.SLG,L1.IVZ,L1.PVH,L1.TPR,L1.NCLH,L1.RL,L1.FTI,L1.DISCA,L1.KSS,L1.XRX,L1.GPS,L1.HRB,L1.UAA,L1.COTY
0,c,aic,5,0.00492,0.01342,0.00498,0.00238,0.00256,0.00102,0.00636,0.0006,0.00238,0.00176,0.0033,0.425819,1.166547,0.057810,0.366859,0.413300,0.415332,0.737451,1.107980,1.070540,1.489626,0.340135,0.519799,0.729145,0.249690,1.757723,2.384511,0.173460,0.418277,2.349222,0.629841,1.406323,1.256391,1.936607,1.909782,4.600287,1.136136,2.497979,0.797371,1.534890,1.128830,0.565797,0.615283,0.448510,0.745636,0.517061,0.399052,0.394416,0.637396,0.569783,0.811080,0.963316,0.689431,1.000856,0.532943,0.220264,0.784741,0.654079,0.867787,0.759656,0.534515,0.663593,1.531352,0.672226,0.466154,1.486465,0.833829,0.728019,0.278250,1.622394,0.615701,

In [14]:
## Here first index represents ticker combinations tried [0-9], 
## 2nd index represents various combinations of trend, IC and maxlags tried [0-59] = [4 trends * 5 ics * 3 maxlags]
final_outputs[0][0]

,date,csi1_pred,csi1,accuracy
4,2019-12-31,1.78,1.86,95.78%
3,2019-12-30,1.71,1.79,95.45%
2,2019-12-27,1.64,1.73,94.81%
1,2019-12-26,1.55,1.61,95.98%
0,2019-12-24,1.45,1.44,100.94%


In [3]:
# Below is some code I was experimenting with to try and "optimize" which tickers to use...
# The function will iterate through VAR multiple times and save the amount of tickers specified in the "step_size" variable
# until it reaches the "generation_size". It will then return those tickers.
# Version 1 searches for the largest coefficients (positive or negative)
# Version 2 searches for coefficients that are most similar in size to CSI's coefficient in the res.summary()
# Version 3 searchs for tickers with the lowest standard error

def ga_best_coefs(tickers, date, variable_of_interest = 'csi1', generation_size = 40, step_size = 3, verbose = False, version = 1):
    
    '''
    Version 1: Largest coefficients
    Version 2: Coefficients closest to CSI
    Version 3: Lowest Std Error in relation to the variable_of_interest
    '''
    
    sample_size = 0
    tickers_used = []
    iteration = 1
    
    while len(tickers_used) < generation_size:
        
        ticker_list = [] + [x for x in tickers_used]
        # Generate ticker list of n size where n == generation_size
        while len(ticker_list) < generation_size:
            ticker_list.append( random.choice(tickers) )   
            ticker_list = list(set(ticker_list))      
            
        # Add variable_of_interest, if necessary    
        if variable_of_interest not in ticker_list:
            ticker_list.append(variable_of_interest)
                     
        # Create subset for only the applicable tickers
        df_subset = df_data.loc[sdate(date):date][ticker_list]

        #Eliminate any tickers that have NaNs as some or all values
        for col in df_subset.columns:    
            # count NaN occurences     
            count_null = df_subset[col].isnull().sum()
            if count_null == 0:
                continue
            else:
                df_subset.drop(columns = col, inplace = True)
                #print('Dropped: ', col)

        # "Difference" the data (statistics term)
        n_obs = 5 ## Beyond 5 results detriorate for majority of predictions

        # Isolate usable columns and ensure csi1 and csi_difference are in the final list
        final_cols = df_subset.columns 

        # Create training and test data
        X_train, X_test = df_subset[final_cols][0:-n_obs], df_subset[final_cols][-n_obs:]
        X_train_transformed = X_train.diff().dropna()    
        
        #return X_train_transformed
        
        # Train model
        model = VAR(X_train_transformed)
        res = model.fit(maxlags=1)

        lag_order = res.k_ar
        # Input data for forecasting
        input_data = X_train_transformed.values[-lag_order:]
        #print(input_data)
        # forecasting
        pred = res.forecast(y=input_data, steps=n_obs)
        pred = (pd.DataFrame(pred, index=X_test.index, columns=X_test.columns + '_pred'))      
        
        # Create dataframe of the results
        if version == 1:
            df_pred = pd.DataFrame(zip(X_train_transformed.columns, abs(res.coefs.mean(axis=2).flatten())), columns = ['ticker', 'coef']).sort_values('coef', ascending = False).reset_index(drop = True)
        
        elif version == 2:
            df_pred = pd.DataFrame(zip(X_train_transformed.columns, (res.coefs.mean(axis=2).flatten())), columns = ['ticker', 'coef']).sort_values('coef', ascending = False).reset_index(drop = True)
            # Sort df_pred by the tickers with the closest coefficient to the coefficient of the variable of interest
            df_pred['diff'] = None
            coef_value = df_pred.set_index('ticker').loc[variable_of_interest]['coef']
            df_pred['diff'] = df_pred.apply(lambda x: abs(x['coef'] - coef_value), axis = 1)
            df_pred = df_pred.sort_values('diff').reset_index(drop = True)    
            
        elif version == 3:
            # Isolate coefficients in relation to the variable of interest
            # c = (pd.DataFrame(res.coefs[0]).loc[0]).values
            # Isolate std_err in relation to the variable of interest
            se = (pd.DataFrame(res.stderr)[variable_of_interest].iloc[1:]).values
            # Create an index based on each ticker name
            index = [x[3:] for x in pd.DataFrame(res.stderr)[variable_of_interest].index[1:]]
            # Create a dataframe using the above information 
            df_c_se = pd.DataFrame(se, index = index, columns = ['std_err'])
            # Sort the dataframe by lowest standard error and drop the variable of interest
            df_pred = df_c_se.sort_values('std_err').drop(variable_of_interest).reset_index().rename(columns = {'index' : 'ticker'})
            
            #return se, index, res, df_pred
        # Save this iterations top tickers to new temp list and ensure variable of interest is included
        tickers_iteration = [variable_of_interest] + [x for x in df_pred.head(step_size * iteration)['ticker']]
        
        # Save the the tickers to the parent list and increase the itration variable by 1
        tickers_used = list(set( [x for x in tickers_used] + [x for x in tickers_iteration] ))
        iteration = iteration + 1
        if verbose == True:
            print('Finished iteration: ', iteration - 1)
        
    return tickers_used

In [ ]:
## problems to solve for

# What does "lag" mean in very elementary terms (like I'm a 5 year old)
# What's the significance of VAR selecting an optimal lag (like, why would 3 maybe more accurate than 6.. what's happening behind the scenese)
# Why is AIC method only selecting 1 as its lag?
# After understanding the above... only then answer the following questions:
# How do we select the best lag? (like, on a human level, what's our mindset and reasoning behind the decision)
# Knowing the above, how do we automate this process?
